# Ingest ADS Notebook Files (Interactive)

![files_to_database](files_to_database.png)

This Notebook will load selected files (from a local or networked file system) into a SQL Server database.

This notebook is designed run _interactively_ and does not require any supporting objects in eltSnap. 

The associated notebook: [Ingest ADS Notebook Files (Project)](ingest_ads_notebook_files_project.ipynb) can be used to Deploy this notebook capability as a runnable _Project_ into the eltSnap database.

> For best results, verify/update the **Notebook Configuration** code cell below, and then use the '**Run all**' option for this Notebook.

## Notebook Configuration

In [42]:
# Configuration Parameters
$FolderLocation = 'C:\ADS\Repository\'   # Update to your Notebook Repository
$ServerInstance = 'localhost'            # Update to your SQL Server Name
$Database = 'eltsnap_v2'

# Set Begin Time
$BegTime = $(Get-Date)
'Beg Time: ' +  $BegTime

# Show Environment
'Run Values:'
'Folder Location: ' + $FolderLocation
'Server: ' + $ServerInstance
'Database: ' +$Database

. {


>> # Configuration Parameters
>> $FolderLocation = 'C:\ADS\Repository\'   # Update to your Notebook Repository
>> $ServerInstance = 'localhost'            # Update to your SQL Server Name
>> $Database = 'eltsnap_v2'
>> 
>> # Set Begin Time
>> $BegTime = $(Get-Date)
>> 'Beg Time: ' +  $BegTime
>> 
>> # Show Environment
>> 'Run Values:'
>> 'Folder Location: ' + $FolderLocation
>> 'Server: ' + $ServerInstance
>> 'Database: ' +$Database
>> }
>> 
Beg Time: 04/16/2021 11:02:17
Run Values:
Folder Location: C:\ADS\Repository\
Server: localhost
Database: eltsnap_v2


## Reset staging table

In [43]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]"

. {


>> Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]"
>> }
>> 


## Load Notebook Files into table: [stg].[files]


In [44]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.ipynb | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"

## Load .SQL Files into table: [stg].[files]

In [45]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.sql | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"

. {


>> Get-ChildItem -Path $FolderLocation -Recurse -Include *.sql | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"
>> }
>> 


## Merge table: [stg].[files] into: [ads].[files]

In [46]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Merge Staged Files];"

. {


>> Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Merge Staged Files];"
>> }
>> 

row_count
---------
       29




## Ingest all Notebooks Files into table: [ads].[notebooks]

In [47]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All Notebooks];"

## Ingest all SQL Files into table: [ads].[sql_files]

In [48]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All SQL Files];"

Invoke-Sqlcmd : Invalid length parameter passed to the LEFT or SUBSTRING function.
Violation of PRIMARY KEY constraint 'PK ads sqlfiles'. Cannot insert duplicate key in object 'ads.sql_files'. The 
duplicate key value is (notebook_snap_basics\content\added_dim_column_names_snowflake_create.sql, source).
Violation of PRIMARY KEY constraint 'PK ads sqlfiles'. Cannot insert duplicate key in object 'ads.sql_files'. The 
duplicate key value is (notebook_snap_basics\content\added_dim_column_names_snowflake_insert.sql, source).
Violation of PRIMARY KEY constraint 'PK ads sqlfiles'. Cannot insert duplicate key in object 'ads.sql_files'. The 
duplicate key value is (notebook_snap_basics\content\build_and_execute_merge_for_dimension_dest_fix_snowflake.sql, 
source).
Violation of PRIMARY KEY constraint 'PK ads sqlfiles'. Cannot insert duplicate key in object 'ads.sql_files'. The 
duplicate key value is (notebook_snap_basics\content\build_and_execute_merge_for_dimension_snowflake.sql, source).
The

## Run Summary

In [49]:
'Beg Time: ' +  $BegTime

$EndTime = $(Get-Date)
'End Time: ' +  $EndTime

$EndTime - $BegTime


Beg Time: 04/16/2021 11:02:17
End Time: 04/16/2021 11:02:19




Days              : 0
Hours             : 0
Minutes           : 0
Seconds           : 2
Milliseconds      : 678
Ticks             : 26786119
TotalDays         : 3.10024525462963E-05


TotalHours        : 0.000744058861111111
TotalMinutes      : 0.0446435316666667
TotalSeconds      : 2.6786119
TotalMilliseconds : 2678.6119



